In [0]:
CREATE OR REFRESH MATERIALIZED VIEW silver.claims
COMMENT "Fully enriched claims data with diagnosis descriptions."
AS
WITH diagnosis_descriptions AS (
  SELECT
    ClaimID,
    collect_list(d.Description) as DiagnosisDescriptions
  FROM (
    SELECT
      ClaimID,
      explode(split(ICD10Codes, ';')) as ICD10Code
    FROM silver.claims_base
  ) c
  LEFT JOIN bronze.diagnosis_ref d ON c.ICD10Code = d.Code
  GROUP BY ClaimID
)
SELECT
  base.*,
  desc.DiagnosisDescriptions
FROM silver.claims_base base
LEFT JOIN diagnosis_descriptions desc ON base.ClaimID = desc.ClaimID;

In [0]:
CREATE OR REFRESH MATERIALIZED VIEW gold.claims_monthly_summary
COMMENT "Materialized view for monthly claims summary by region and type."
AS
SELECT
  window(ClaimDate, "30 days") as month_window,
  MemberRegion,
  ClaimType,
  COUNT(ClaimID) as number_of_claims,
  SUM(Amount) as total_amount
FROM
  silver.claims  
GROUP BY
  window(ClaimDate, "30 days"),
  MemberRegion,
  ClaimType;


In [0]:
CREATE OR REFRESH MATERIALIZED VIEW gold.fraud_alerts
COMMENT "Materialized view for claims flagged with a high potential fraud score."
AS
WITH scored_claims AS (
  SELECT
    *,
    CASE WHEN Amount > 4500 THEN 50 ELSE 0 END +
    CASE WHEN dayofweek(ClaimDate) IN (1, 7) THEN 25 ELSE 0 END +
    CASE WHEN Status = 'Rejected' THEN 10 ELSE 0 END as fraud_score
  FROM
    silver.claims  
)
SELECT * FROM scored_claims
WHERE fraud_score > 5;